In [1]:
from myria import *
import numpy
import json
from myria.connection import MyriaConnection
from myria.relation import MyriaRelation
from myria.udf import MyriaPythonFunction
from raco.types import STRING_TYPE, BOOLEAN_TYPE, LONG_TYPE, BLOB_TYPE

#create connection
connection = MyriaConnection(rest_url='http://localhost:8753',execution_url='http://localhost:8080')

In [2]:
#register Python functions
outType= "LONG_TYPE"
def pyIsPrime(dt):
    import math
    n = dt[0][0]
    if n % 2 == 0 and n > 2: 
        return False
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return 0
    return 1

MyriaPythonFunction(pyIsPrime, outType).register()

In [ ]:
connection.get_functions()

In [ ]:
connection.get_function('pyIsPrime')

In [ ]:
q = MyriaQuery.submit(""" T1 = scan(TwitterK);
T2 = [from T1 emit pyIsPrime(T1.src) as isPrime, T1.src, T1.dst];
primeCount = [from T2 emit  sum(T2.isPrime) as prime];
store( isPrime, TwitterK_isPrime);""", connection=connection)
q.status

In [ ]:
# 1: Download as a Python dictionary
d = MyriaRelation("primeCount").to_dict()
print 'Number of Users with id that is prime: %s' % d[0]['prime']

In [ ]:
#Register a python UDA 
def pyAdd(dt):
    tuplist = dt
    retval = None
    for i in tuplist:
        if retval is None:
            retval = i[0]
        else:
            retval = retval+i[0]
    return retval

MyriaPythonFunction(pyAdd, BLOB_TYPE).register()

def pyMean(dt):
    _sum= dt[0][0]
    _count = dt[0][1]
    return _sum/_count

MyriaPythonFunction(pyMean, BLOB_TYPE).register()

In [ ]:
q = MyriaQuery.submit(""" 
uda average_image(img) {
   [EMPTY_BLOB as val, 0 as cnt];
   [pyAdd(img), cnt + 1];
   [pyMean(img,cnt)];
};
t = scan(public:adhoc:raw);
results = [from t emit t.subjid average_image(t.img) as mean_image];
STORE(results, results);
""", connection=connection)
q.status